# Desafio Laboratório Prático Final
###### Tópicos em Banco de Dados
======

Passos realizados no desafio proposto:<br>
1- Instalação das dependências necessárias para o projeto, como o pymongo e o elasticsearch.<br>
2- Configuração dos avisos no projeto.<br>
3- Conversão de CSV para BSON.<br>


In [1]:
# instalando dependências necessárias
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 1.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.5/905.5 kB 2.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 700.7 kB/s eta 0:00:00 0:00:01


## Configurando bibliotecas e estruturas padrão para o projeto

In [8]:
# bibliotecas necessárias
import datetime
import pymongo
from pymongo import MongoClient
# bibliotecas de aprendizado de máquina e mineração de dados
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import re
import nltk
from nltk.corpus import stopwords
import tqdm
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import string

## Configuração de Avisos

In [3]:
from warnings import filterwarnings
filterwarnings('ignore')
pd.set_option('display.max_colwidth',100)

## Conversão do arquivo CSV para BSON (MongoDB)

In [7]:
data = pd.read_csv("df_file.csv", sep=",")
print(data.head(2))


                                                                                                  Text  \
0  Budget to set scene for election\n \n Gordon Brown will seek to put the economy at the centre of...   
1  Army chiefs in regiments decision\n \n Military chiefs are expected to meet to make a final deci...   

   Label  
0      0  
1      0  


## Interação com o MongoDB

In [5]:
mongoClient = MongoClient('mongodb://localhost:27017/');

db = mongoClient.desafio

